In [1]:
import pickle
policy_model = "Qwen2.5-1.5B-Instruct"
reward_model = "QRM-Llama3.1-8B-v2"
window = 100
n_prompts = 16
n_gens = 16
seed = 42   

filename = f"{policy_model}_{reward_model}_window{window}_prompt{n_prompts}_gen{n_gens}_seed{seed}.pkl"

with open(filename, "rb") as f:
    reward_outputs = pickle.load(f)

In [2]:
len(reward_outputs)

100

In [3]:
# step: training step
# input_ids: input ids of the prompt + completion, shape: (n_prompts, n_gens, seq_len)
# reward_quantiles: quantiles of the reward, shape: (n_prompts, n_gens, 19)
# reward_detailed: detailed reward ['helpsteer-helpfulness','helpsteer-correctness','helpsteer-coherence', 'helpsteer-complexity','helpsteer-verbosity'], shape: (n_prompts, n_gens, 5)
# reward_aggregated: aggregated reward, shape: (n_prompts, n_gens)
# hidden_states: hidden states of the reward model, seq_len reduced, shape: (n_prompts, n_gens, hidden_size)
reward_outputs[0].keys()

dict_keys(['step', 'input_ids', 'reward_quantiles', 'reward_detailed', 'reward_aggregated', 'hidden_states'])

In [4]:
reward_outputs[0]['input_ids'].shape

(16, 16, 1188)

In [6]:
reward_outputs[0]['hidden_states'].shape

(16, 16, 4096)

In [7]:
reward_outputs[0]['reward_aggregated'].shape

(16, 16)